In [1]:
import os
import pprint

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from sklearn.model_selection import train_test_split

os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"

In [2]:
dataset_path = "C:/Users/user/Desktop/[사이버 보안 챌린지 2020] 자동차 해킹공격방어_데이터셋/학습용 데이터셋"
print(os.listdir(dataset_path))

['Cybersecurity_Car_Hacking_D_training-0.csv', 'Cybersecurity_Car_Hacking_D_training-1.csv', 'Cybersecurity_Car_Hacking_D_training-2.csv', 'Cybersecurity_Car_Hacking_S_training-0.csv', 'Cybersecurity_Car_Hacking_S_training-1.csv', 'Cybersecurity_Car_Hacking_S_training-2.csv']


In [3]:
D_normal_dfs = []
D_normal_dfs.append(pd.read_csv(os.path.join(dataset_path,"Cybersecurity_Car_Hacking_D_training-0.csv")))
D_normal_dfs=pd.concat(D_normal_dfs)
D_normal_dfs.head()

,Timestamp,Arbitration_ID,DLC,Data,Class
0,1.597708e+09,260,8,06 25 05 30 FF CF 71 55,Normal
1,1.597708e+09,329,8,4A C5 7E 8C 31 2D 01 10,Normal
2,1.597708e+09,38D,8,00 00 49 00 90 7F FE 01,Normal
3,1.597708e+09,420,8,50 1E 00 C8 FC 4F 6A 00,Normal
4,1.597708e+09,421,8,FE 07 00 FF E3 7F 00 52,Normal


# LSTM Autoencoder
Sequence 데이터에 Encoder-Decoder LSTM 아키텍처를 적용하여 구현한 모델


In [4]:
can_id = D_normal_dfs["Arbitration_ID"]
sequences = can_id.apply(lambda x: int(str(x), 16))

In [5]:
sequences=np.array(sequences)

In [9]:
from tensorflow.keras import Model, models, layers, optimizers, utils

#semi_sequences1 = np.array([100,200,300,400,500,600,700,800,900])
# ===== reshape input into [samples, timesteps, features] =====
n_in = len(sequences)
sequences = sequences.reshape((1, n_in, 1))
    #n_in = len(semi_sequences)
    #semi_sequences = np.expand_dims(semi_sequences, axis=0)
    #semi_sequences = semi_sequences.reshape((1, n_in, 1))
    #semi_sequences = np.expand_dims(semi_sequences, axis=2)
# ===== prepare output sequence =====
  #semi_sequences_out = semi_sequences[:, 1:, :]
sequences_out = sequences[:, 1:, :]
n_out = n_in - 1

# ===== define model =====
model = models.Sequential()
model.add(layers.LSTM(100, activation='relu', input_shape=(n_in, 1)))
model.add(layers.RepeatVector(n_out))
model.add(layers.LSTM(100, activation='relu', return_sequences=True))
model.add(layers.TimeDistributed(layers.Dense(1)))
model.compile(optimizer='adam', loss='mse')


ValueError: cannot reshape array of size 179346 into shape (1,1,1)

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 100)               40800     
_________________________________________________________________
repeat_vector (RepeatVector) (None, 179345, 100)       0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 179345, 100)       80400     
_________________________________________________________________
time_distributed (TimeDistri (None, 179345, 1)         101       
Total params: 121,301
Trainable params: 121,301
Non-trainable params: 0
_________________________________________________________________


In [7]:
# ===== fit model =====
model.fit(semi_sequences, semi_sequences_out, epochs=1000, verbose=0)
# ===== predict =====
yhat = model.predict(semi_sequences)

In [7]:
sequences.shape[1]

179346